# Phenomenological Synaptic Models

In [ ]:
%reset -f
import numpy as np
import brainpy as bp
import brainpy.math as bm
import matplotlib.pyplot as plt

### COBA

Given the synaptic conductance, the COBA model outputs the post-synaptic current with

$$
I_{syn}(t) = g_{\mathrm{syn}}(t) (E - V(t))
$$


### CUBA

Given the conductance, this model outputs the post-synaptic current with a identity function:

$$
I_{\mathrm{syn}}(t) = g_{\mathrm{syn}}(t)
$$

## ``brainpy.dyn.ProjAlignPreMg2``

Synaptic projection which defines the synaptic computation with the dimension of presynaptic neuron group.


```
brainpy.dyn.ProjAlignPreMg2(
   pre, 
   delay,
   syn,  
   comm, 
   out, 
   post
)
```

- ``pre (JointType[DynamicalSystem, AutoDelaySupp])``: The pre-synaptic neuron group.
- ``delay (Union[None, int, float])``: The synaptic delay.
- ``syn (ParamDescInit)``: The synaptic dynamics.
- ``comm (DynamicalSystem)``: The synaptic communication.
- ``out (ParamDescInit)``: The synaptic output.
- ``post (DynamicalSystem)`` The post-synaptic neuron group.


![](figs/align_pre.png)


## Dual Exponential Model

The dual exponential synapse model, also named as *difference of two exponentials model*, is given by:

$$
g_{\mathrm{syn}}(t)=\bar{g}_{\mathrm{syn}} \frac{\tau_{1} \tau_{2}}{\tau_{1}-\tau_{2}}\left(\exp \left(-\frac{t-t_{0}}{\tau_{1}}\right)-\exp \left(-\frac{t-t_{0}}{\tau_{2}}\right)\right)
$$

where $\tau_1$ is the time constant of the decay phase, $\tau_2$ is the time constant of the rise phase, $t_0$ is the time of the pre-synaptic spike, $\bar{g}_{\mathrm{syn}}$ is the maximal conductance.

The corresponding differential equation:

$$
\begin{aligned}
&g_{\mathrm{syn}}(t)=\bar{g}_{\mathrm{syn}} g \\
&\frac{d g}{d t}=-\frac{g}{\tau_{\mathrm{decay}}}+h \\
&\frac{d h}{d t}=-\frac{h}{\tau_{\text {rise }}}+ \delta\left(t_{0}-t\right),
\end{aligned}
$$

The alpha function is retrieved in the limit when both time constants are equal.

# VN -> CbX

In [ ]:
duration = 0.5 # unit s
fr = 5

Synapse: Dual Exponential

In [ ]:
class DualExpSparseCOBA(bp.Projection):
  def __init__(self, pre, post, delay, prob, g_max, tau_decay, tau_rise, E):
    super().__init__()
    
    self.proj = bp.dyn.ProjAlignPreMg2(
      pre=pre, 
      delay=delay, 
      syn=bp.dyn.DualExpon.desc(pre.num, tau_decay=tau_decay, tau_rise=tau_rise),
      comm=bp.dnn.CSRLinear(bp.conn.FixedProb(prob, pre=pre.num, post=post.num), g_max),
      out=bp.dyn.COBA(E=E),
      post=post, 
    )

Generate input spike times

In [ ]:
def Poisson_sptrain():
    n = np.random.poisson(fr * duration)  # 计算时间段T内的事件总数（泊松分布）
    spike_times = np.sort(np.random.uniform(0, duration, n))  # 在[0,T]内均匀生成n个事件时间点并排序
    spike_times = spike_times * 1000  # unit ms
    isi = np.diff(spike_times)  # 计算事件间隔时间（Inter-Spike Interval）unit ms
    return spike_times, isi  # 返回事件时间序列和间隔序列

Connection: SpTrain -> Synapse -> LIF

In [ ]:
class VN_CbX_Net(bp.DynSysGroup):
  def __init__(self, sptimes, E=0.):
    super().__init__()
    
    #self.pre = bp.dyn.SpikeTimeGroup(1, indices=(0, 0), times=(0., 100.))  # single spike input
    self.pre = bp.dyn.SpikeTimeGroup(1, times=sptimes, indices=[0] * len(sptimes))  # spike train input
    self.post = bp.dyn.LifRef(1, V_rest=-60., V_th=-50., V_reset=-60., tau=20., tau_ref=5.,
                              V_initializer=bp.init.Constant(-60.))
    '''
    # 默认 normal
    self.syn = DualExpSparseCOBA(self.pre, self.post, delay=None, prob=1., g_max=1., 
                                 tau_decay=5., tau_rise=1., E=E)  
    
    # Syt2 KO 上升更慢  衰减也更慢（囊泡释放更慢）  峰值更低(囊泡释放更少)
    # V1 gmax' = 1/10gmax = 0.1, V2 gmax' = 0.2, V3 gmax' = 0.3
    self.syn = DualExpSparseCOBA(self.pre, self.post, delay=None, prob=1., 
                                tau_rise=12., tau_decay=20., g_max=0.1, E=E)  # 
    '''
    # junqiang parameters 会让postV产生波形抖动，符合patch结果
    self.syn = DualExpSparseCOBA(self.pre, self.post, delay=None, prob=1., 
                                tau_rise=20., tau_decay=0.5, g_max=0.8, E=E)
    
    # Syt2 overexpression 上升更快  衰减也更快（囊泡释放更快）  峰值更高(囊泡释放增加)
    #self.syn = DualExpSparseCOBA(self.pre, self.post, delay=None, prob=1., 
                                #tau_rise=0.5, tau_decay=3., g_max=1.7, E=E)   
    
  def update(self):
    self.pre()
    self.syn()
    self.post()
    
    # monitor the following variables
    conductance = self.syn.proj.refs['syn'].g
    current = self.post.sum_inputs(self.post.V)
    return conductance, current, self.post.V

In [ ]:
def run_a_net_plot(net):
  indices = np.arange(10000) # for single spike 500ms
  indices = np.arange(duration*10000)   # for spike train
  conductances, currents, potentials = bm.for_loop(net.step_run, indices, progress_bar=True)
  ts = indices * bm.get_dt()
  
  # --- similar to: 
  # runner = bp.DSRunner(net)
  # conductances, currents, potentials = runner.run(100.)
  
  fig, gs = bp.visualize.get_figure(1, 3, 3.5, 4)
  fig.add_subplot(gs[0, 0])
  plt.plot(ts, conductances)
  plt.title('Syn conductance')
  fig.add_subplot(gs[0, 1])
  plt.plot(ts, currents)
  plt.title('Syn current')
  fig.add_subplot(gs[0, 2])
  plt.plot(ts, potentials)
  plt.title('Post V')
  plt.show()
  return potentials

In [ ]:
def run_a_net(net):
  indices = np.arange(duration*10000)   # for spike train
  conductances, currents, potentials = bm.for_loop(net.step_run, indices, progress_bar=True)
  ts = indices * bm.get_dt()
  return potentials

In [ ]:
def find_trig_spike_time(arr):
    #找到所有满足前值 > -50.8 且后值 = -60 的连续元素对
    mask = (arr[:-1] > -50.8) & (arr[1:] == -60) # 通过切片操作，隐式比较了每一对相邻元素
    indices = np.where(mask)[0] / 10  # 换算为ms
    return indices

def plot_isi(isi,name,cutoff_distr):
    isi = isi[(isi > 0.001) & (isi <= cutoff_distr)]
    plt.figure(figsize=(10, 4))
    plt.hist(isi, bins=100, color='dodgerblue', alpha=0.7)
    plt.xlabel('Inter-spike Interval (ms)')
    plt.ylabel('Counts')
    plt.title(f'{name} ISI Distribution')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

Syt2KO VN -> CbX Excitatory

In [ ]:
VN_sptimes, VN_isi = Poisson_sptrain()  # input_sptimes unit ms
CbX_PostV = run_a_net_plot(VN_CbX_Net(sptimes = VN_sptimes, E=0.))  # Excitatory DualExpon synapse model
CbX_PostV_value = CbX_PostV.value  # 获取 Array 对象
CbX_PostV_flat = CbX_PostV_value.flatten()  # 平铺成一维数组
np.set_printoptions(threshold=np.inf)

CbX_sptimes = find_trig_spike_time(CbX_PostV_flat)  # unit ms
CbX_isi = np.diff(CbX_sptimes)

In [ ]:
plt.figure(figsize=(45, 2))
plt.eventplot([VN_sptimes, CbX_sptimes], colors=['b', 'r'], lineoffsets=[2, 1], linelengths=0.8)
plt.yticks([1, 2], ['CbX', 'VN'])
plt.xlabel('Time (ms)')
plt.title('Raster Plot of Input and Output Spike Times')
plt.tight_layout()
plt.savefig('/home/zhangyuhao/Desktop/Result/ET/Modeling/Raster.png', transparent=True, dpi=500)

In [ ]:
plot_isi(VN_isi,'input',100)
plot_isi(CbX_isi,'output',100)  #尽量让这个分布密度密一些，峰值更大一些，峰更靠右一些

# CbX -> DCN

create three neurons with renewal process ISI <br>
A single neuron in the deep cerebellar nuclei (DCN) receives input from a population of approximately 40 Purkinje cells (Kathellen Cullen)<br>
A single PC is estimated to innervate approximately 30–40 DCN neurons and in turn, each DCN neuron, receives projections from about 600–900 PC (Chan-Palay, 1973a ; Mezey et al., 1977 ; Palkovits et al., 1977 ) <br>但并不是所有的PC都会发放

循环生成15个VN->CbX Syt2KO神经元的spike train 循环生成5个正常神经元的spike train

In [ ]:
syt2ko_num = 15  # synchrony neuron number syt2ko
normal_num = 5   # synchrony neuron number 正常神经元 
neuron_num = syt2ko_num + normal_num

# 初始化存放所有发放事件的列表
all_events = []  # 格式：(spike_time, neuron_id)

# 生成syt2ko神经元的发放事件
for i in range(syt2ko_num):
    VN_sptimes, _ = Poisson_sptrain()  # 忽略ISI数据
    CbX_PostV = run_a_net(VN_CbX_Net(sptimes = VN_sptimes, E=0.))
    CbX_PostV_value = CbX_PostV.value  # 获取 Array 对象
    CbX_PostV_flat = CbX_PostV_value.flatten()  # 平铺成一维数组
    CbX_Syt2ko_sptimes = find_trig_spike_time(CbX_PostV_flat)  # unit ms

    # 为当前神经元的每个发放时间添加事件
    for t in CbX_Syt2ko_sptimes:
        all_events.append((t, i))  # 记录(时间, 神经元ID)

# 生成正常神经元的发放事件
for j in range(normal_num):
    CbX_normal_sptimes, _ = Poisson_sptrain()  # 忽略ISI数据
    neuron_id = j + syt2ko_num  
    
    # 为当前神经元的每个发放时间添加事件
    for t in CbX_normal_sptimes:
        all_events.append((t, neuron_id))

# 按发放时间排序
all_events.sort(key=lambda x: x[0])  # 根据spike_time排序

# 提取排序后的发放时间和对应神经元ID
spike_times = [event[0] for event in all_events]
spike_cluster = [event[1] for event in all_events]

output to LIF

In [ ]:
class CbX_DCN_Net(bp.DynSysGroup):
  def __init__(self, E=0.):
    super().__init__()
    # times is spike times of each spike, indices is unit id of each spike
    self.pre = bp.dyn.SpikeTimeGroup(neuron_num, times=spike_times, indices=spike_cluster)  # spike train input
    self.post = bp.dyn.LifRef(1, V_rest=-60., V_th=-50., V_reset=-60., tau=20., tau_ref=5.,
                              V_initializer=bp.init.Constant(-60.))

    self.syn = DualExpSparseCOBA(self.pre, self.post, delay=None, prob=1., g_max=1., 
                                 tau_decay=5., tau_rise=1., E=E)             
    
  def update(self):
    self.pre()
    self.syn()
    self.post()
    
    # monitor the following variables
    conductance = self.syn.proj.refs['syn'].g
    current = self.post.sum_inputs(self.post.V)
    return conductance, current, self.post.V

all neurons projecting from the cerebellar cortex to the deep cerebellar nuclei are inhibitory neurons

In [ ]:
DCN_PostV = run_a_net(CbX_DCN_Net(E=-35.))
DCN_PostV_value = DCN_PostV.value  # 获取 Array 对象
DCN_PostV_flat = DCN_PostV_value.flatten()  # 平铺成一维数组
DCN_sptimes = find_trig_spike_time(DCN_PostV_flat)  # unit ms
DCN_isi = np.diff(DCN_sptimes)

In [ ]:
plt.figure(figsize=(100, 2))
plt.eventplot([CbX_sptimes, DCN_sptimes], colors=['b', 'r'], lineoffsets=[2, 1], linelengths=0.8)
plt.yticks([1, 2], ['DCN', 'CbXSyt2ko'])
plt.xlabel('Time (ms)')
plt.title('Raster Plot of Input and Output Spike Times')
plt.tight_layout()

In [ ]:
plot_isi(CbX_isi,'CbX')
plot_isi(DCN_isi,'DCN')

# DCN inner network

In [ ]:
DCN_inner_neunum = 3
DCN_inner_sptimes = np.repeat(output_sptimes2, DCN_inner_neunum)
DCN_inner_indices = list(range(DCN_inner_neunum)) * len(output_sptimes2)

In [ ]:
class SimpleNet6(bp.DynSysGroup):
  def __init__(self, E=0.):
    super().__init__()
    
    self.pre = bp.dyn.SpikeTimeGroup(DCN_inner_neunum, times=DCN_inner_sptimes, indices=DCN_inner_indices)  # spike train input
    self.post = bp.dyn.LifRef(1, V_rest=-60., V_th=-50., V_reset=-60., tau=20., tau_ref=5.,
                              V_initializer=bp.init.Constant(-60.))

    self.syn = DualExpSparseCOBA(self.pre, self.post, delay=None, prob=1., g_max=1., 
                                 tau_decay=5., tau_rise=1., E=E)  
    
  def update(self):
    self.pre()
    self.syn()
    self.post()
    
    # monitor the following variables
    conductance = self.syn.proj.refs['syn'].g
    current = self.post.sum_inputs(self.post.V)
    return conductance, current, self.post.V

Deep cerebellar nuclei (DCN) have projection neurons that contribute to the cerebellum's output <br>
These projections can be both excitatory and inhibitory

In [ ]:
PostV3 = run_a_net(SimpleNet6(E=0.))
value_array3 = PostV3.value  # 获取 Array 对象
flattened_arr3 = value_array3.flatten()  # 平铺成一维数组
np.set_printoptions(threshold=np.inf)

output_sptimes3 = find_trig_spike_time(flattened_arr3)  # unit ms
output_isi3 = np.diff(output_sptimes3)

In [ ]:
plt.figure(figsize=(60, 2))
plt.eventplot([output_sptimes2, output_sptimes3], colors=['b', 'r'], lineoffsets=[2, 1], linelengths=0.8)
plt.yticks([1, 2], ['Output', 'Input'])
plt.xlabel('Time (ms)')
plt.title('Raster Plot of Input and Output Spike Times')
plt.tight_layout()
plt.savefig('/home/zhangyuhao/Desktop/Result/ET/Modeling/Raster.png', transparent=True, dpi=500)

In [ ]:
cutoff_distr = 100
def plot_isi(isi,name):
    isi = isi[(isi > 0.001) & (isi <= cutoff_distr)]
    plt.figure(figsize=(10, 4))
    plt.hist(isi, bins=100, color='dodgerblue', alpha=0.7)
    plt.xlabel('Inter-spike Interval (ms)')
    plt.ylabel('Counts')
    plt.title(f'{name} ISI Distribution')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

plot_isi(output_isi2,'input')
plot_isi(output_isi3,'output')

## Problem of Phenomenological Synaptic Models

A significant limitation of the simple waveform description of synaptic conductance is that it does not capture the actual behavior seen at many synapses when trains of action potentials arrive. 

A new release of neurotransmitter soon after a previous release should not be expected to contribute as much to the postsynaptic conductance due to saturation of postsynaptic receptors by previously released transmitter and the fact that some receptors will already be open.

In [ ]:
class SimpleNet5(bp.DynSysGroup):
  def __init__(self, freqs=10.):
    super().__init__()
    self.pre = bp.dyn.PoissonGroup(1, freqs=freqs)
    self.post = bp.dyn.LifRef(1, V_rest=-60., V_th=-50., V_reset=-60., tau=20., tau_ref=5.,
                              V_initializer=bp.init.Constant(-60.))
    self.syn = DualExpSparseCOBA(self.pre, self.post, delay=None, prob=1., g_max=1., 
                                 tau_decay=5., tau_rise=1., E=0.)
    
  def update(self):
    self.pre()
    self.syn()
    self.post()
    return self.syn.proj.refs['syn'].g, self.post.V

In [ ]:
def compare(freqs):
  fig, _ = bp.visualize.get_figure(1, 1, 4.5, 6.)
  for freq in freqs:
    net = SimpleNet5(freqs=freq)
    indices = np.arange(1000)  # 100 ms
    conductances, potentials = bm.for_loop(net.step_run, indices, progress_bar=True)
    plt.plot(indices * bm.get_dt(), conductances, label=f'{freq} Hz')
  plt.legend()
  plt.ylabel('g')
  plt.show()


compare([10., 100., 1000., 8000.])